In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import imodels
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    LabelEncoder,
    OneHotEncoder,
)
from sklearn.impute import SimpleImputer

## Load the dataset

In [2]:
df = pd.read_csv("./data/breast_cancer/breast_cancer.csv")

In [3]:
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [4]:
df["Status"].value_counts()

Status
Alive    3408
Dead      616
Name: count, dtype: int64

## Preprocessing

In [5]:
encoder = LabelEncoder()
for col in df.select_dtypes("object"):
    df[col] = encoder.fit_transform(df[[col]])

/home/lcorbucci/personalized_explanations/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lcorbucci/personalized_explanations/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lcorbucci/personalized_explanations/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/lcorbucci/personalized_explanations/.venv/lib/python3.10/site-packages/sklearn/prepro

In [6]:
x = df.drop(columns="Status")
y = df.loc[:, "Status"]
x.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months
0,68,2,1,0,0,0,1,3,1,4,1,1,24,1,60
1,50,2,1,1,1,2,0,2,1,35,1,1,14,5,62
2,58,2,0,2,2,4,0,2,1,63,1,1,14,7,75
3,58,2,1,0,0,0,1,3,1,18,1,1,2,1,84
4,47,2,1,1,0,1,1,3,1,41,1,1,3,1,50


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=41, stratify=y)

In [8]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
x_train

array([[0.69230769, 1.        , 0.        , ..., 0.01666667, 0.        ,
        0.63207547],
       [0.79487179, 1.        , 0.25      , ..., 0.63333333, 0.        ,
        0.70754717],
       [0.33333333, 1.        , 0.25      , ..., 0.28333333, 0.28888889,
        0.60377358],
       ...,
       [0.20512821, 1.        , 0.25      , ..., 0.35      , 0.28888889,
        0.80188679],
       [0.30769231, 1.        , 0.25      , ..., 0.08333333, 0.        ,
        0.6509434 ],
       [0.33333333, 1.        , 0.75      , ..., 0.16666667, 0.        ,
        0.51886792]])

In [10]:
y_test.value_counts()

Status
0    682
1    123
Name: count, dtype: int64

# Train Logistic regressor

In [11]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9031055900621118


In [12]:
# Majority classifier class 0
y_pred = np.zeros(y_test.shape)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8472049689440994
